# 读取文件夹内数据

In [1]:
import os 
import numpy as np
import pandas as pd

In [2]:
# 获取数据文件夹下，各个表格名字
list_dir=os.listdir('数据')

In [3]:
# 各个表格数据转化为pandas数据，共8个表格，转化为all_data=[df0,df1,df2,df3,df4,df5,df6,df7]
all_data=[]
for k,i in enumerate (list_dir):
    locals()['df' + str(k)]=pd.read_excel(r'数据/%s'%(i))
#     locals()['df' + str(k)]['merge']=locals()['df' + str(k)]['year']+str(locals()['df' + str(k)]['code'])
    all_data.append(locals()['df' + str(k)])

PermissionError: [Errno 13] Permission denied: '数据/~$OFDI公司财务指标.xlsx'

In [ ]:
# 合并各个表格数据
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on=['code','year']), all_data)

In [ ]:
# 所有空值替换为0
df_final.fillna(value=0,inplace=True)

In [ ]:
# 计算收入，用研发投入除以研发占收入比重
df_final['income']=df_final['RDSpendSum']*100/df_final['RDSpendSumRatio']
# 无法除的替换为0 ，inf>>0
df_final.replace(np.inf, 0,inplace=True)
# for i in df_final['RDSpendSumRatio']:
    

In [ ]:
# 迎合性判别
cond1=((df_final.income<5000*10000) &df_final.RDSpendSumRatio.isin([6,7]))
cond2=((df_final.income.isin([5000*10000,20000*10000])) &df_final.RDSpendSumRatio.isin([4,5]))
cond3=((df_final.income>20000*10000) &df_final.RDSpendSumRatio.isin([3,4]))
# cond1.value_counts() 

In [ ]:
# 看看多少满足迎合性条件
cond1.value_counts() ,cond2.value_counts() ,cond3.value_counts() 

In [ ]:
# 迎合性为1，否则为0
df_final['yinghe']=0
df_final.loc[cond1,'yinghe']=1
df_final.loc[cond2,'yinghe']=1
df_final.loc[cond3,'yinghe']=1
df_final.yinghe.value_counts()

# 挑选指标

In [ ]:
# 相关系数

s=df_final.corr().loc[['yinghe']].T
s

In [ ]:
s.describe()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbdt=GradientBoostingClassifier()

In [ ]:
# 只要各种数据指标，规律为列名F开头的
var_name=df_final.columns
var_zhibiao=[]
for i in var_name:
    if i[0]=='F':
        var_zhibiao.append(i)
        

In [ ]:
X=df_final[var_zhibiao]
y=df_final['yinghe']

In [ ]:
# 数据标准化处理

In [ ]:
X.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler 
standard=StandardScaler()

# data['s_amount']=standard.fit_transform(data[['Amount']])

In [ ]:
for i in X.columns:
    X[i]=standard.fit_transform(X[[i]])

In [ ]:
X.describe()

In [ ]:
# 梯度决策树
gbdt.fit(X,y)

In [ ]:
gbdt.feature_importances_

In [ ]:
argsort=gbdt.feature_importances_.argsort()
argsort

In [ ]:
np.arange(X.columns.size),gbdt.feature_importances_[argsort],X.columns[argsort]

In [ ]:
X.columns[argsort][-50:]

In [ ]:
# 取最重要的50个变量
X=df_final[X.columns[argsort][-50:]]

In [ ]:
X.describe()

In [ ]:
# 标准化
for i in X.columns:
    X[i]=standard.fit_transform(X[[i]])

In [ ]:
X.describe()

In [ ]:
# 过采样
from imblearn.over_sampling import SMOTE
smote=SMOTE()
X1,y1=smote.fit_resample(X,y)

In [ ]:
X1.shape,X.shape

In [ ]:
y1.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X1,y1,test_size=0.2)

In [ ]:
# logistic
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()
lg.fit(X_train,y_train)

In [ ]:
lg.score(X_test,y_test)

In [ ]:
y_=lg.predict(X_test)
y_

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm=confusion_matrix(y_test,y_)
plt.rc('figure',figsize=(5,5))
plt.matshow(cm,cmap = plt.cm.Reds) # 背景颜色
plt.colorbar() # 颜色标签
 
# 内部添加图例标签
for x in range(len(cm)):
    for y in range(len(cm)):
        plt.annotate(cm[x,y], xy = (x,y), horizontalalignment = 'center', verticalalignment = 'center')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Cart_train')
plt.show()

In [ ]:
# tf

In [ ]:
import tensorflow as tf


In [ ]:

X_train = np.array(X1)
y_train = np.array(y1)

In [ ]:
# 随机打乱数据（因为原始数据是顺序的，顺序不打乱会影响准确率）
# seed: 随机数种子，是一个整数，当设置之后，每次生成的随机数都一样
np.random.seed(42)  # 使用相同的seed，保证输入特征和标签一一对应
np.random.shuffle(X_train)
np.random.seed(42)
np.random.shuffle(y_train)
tf.random.set_seed(42)

In [ ]:
#  设定模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())
])
# 设置学习率
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

# 保存参数，断点续训
checkpoint_save_path = "./checkpoint/train.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)
# 设定模型训练的超参数，验证集比例
history = model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.2, validation_freq=1,
                    callbacks=[cp_callback])

model.summary()

In [ ]:
import matplotlib.pyplot as plt
# 显示训练集和验证集的acc和loss曲线
history_dict = history.history
history_dict.keys()

acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# 预测值X_test 
predictions = model.predict(X_test)

In [ ]:
pre_y=tf.constant(predictions)
pre_y

In [ ]:
np.array(pre_y)

In [ ]:
np.array(pre_y).argsort()

In [ ]:
y_p_2and=np.array(pre_y)
y_p_1max=np.array(pre_y).argsort()[:,1]

In [ ]:
df_y_p_2and=pd.DataFrame(y_p_2and,columns=['p_0','p_1'])
df_y_p_2and

In [ ]:
df_y_p_2and['p_max']=y_p_1max
df_y_p_2and

In [ ]:
df_y_p_2and['p_max'].value_counts()

In [ ]:
df_y_p_2and['p_max70']=df_y_p_2and['p_1']>0.7
df_y_p_2and

In [ ]:
df_y_p_2and['p_max70'].value_counts()